# Ground state solvers

## Introduction

<img src="aux_files/H2_gs.png" width="200">

In this tutorial we are going to discuss the ground state calculation interface of Qiskit Nature. The goal is to compute the ground state of a molecular Hamiltonian. This Hamiltonian can for example be electronic or vibrational. To know more about the preparation of the Hamiltonian, check out the [Electronic structure](01_electronic_structure.ipynb) and [Vibrational structure tutorials](02_vibrational_structure.ipynb). 

> It should be said, that in the electronic case, we are actually computing purely the **electronic** part. When using the Qiskit Nature stack as presented in this tutorial, the _nuclear repulsion energy_ will be added automatically, to obtain the **total** ground state energy.

The first step is to define the molecular system. In the following we ask for the electronic part of a hydrogen molecule.

In [14]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H; O 1 1.08; H 2 1.08 1 104.5",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

We will also be sticking to the Jordan-Wigner mapping. To learn more about the various mappers available in Qiskit Nature, check out the [Qubit Mappers tutorial](06_qubit_mappers.ipynb).

In [15]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

## The Solver

After these steps, we need to define a solver. The solver is the algorithm through which the ground state is computed. 

Let's first start with a purely classical example: the `NumPyMinimumEigensolver`. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms. 

In [16]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithm works by exchanging information between a classical and a quantum computer as depicted in the following figure.

<img src="aux_files/vqe.png" width="600">

Let's initialize a VQE solver.

In [17]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

To define the VQE solver one needs three essential elements:

1. An Estimator primitive: these were released as part of Qiskit Terra 0.22. To learn more about primitives, check out [this resource](https://docs.quantum.ibm.com/api/qiskit/primitives).
2. A variational form: here we use the Unitary Coupled Cluster (UCC) ansatz (see for instance [Physical Review A 98.2 (2018): 022322]). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected. We also prepend the `UCCSD` variational form with a `HartreeFock` initial state, which initializes the occupation of our qubits according to the problem which we are trying solve.
3. An optimizer: this is the classical piece of code in charge of optimizing the parameters in our variational form. See [the corresponding documentation](https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.optimizers.html) for more information.

One could also use any available ansatz / initial state or even define one's own. For instance,

In [18]:
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(Estimator(), tl_circuit, SLSQP())

## The calculation and results

We are now ready to put everything together to compute the ground-state of our problem.
Doing so requires us to wrap our `mapper` and quantum algorithm into a single `GroundStateEigensolver` like so:

In [19]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

This will now take of the entire workflow:
1. generating the second-quantized operators stored in our problem (here referred to as `es_problem`)
2. mapping (and potentially reducing) the operators in the qubit space
3. running the quantum algorithm on the Hamiltonian qubit operator
4. once converged, evaluating the additional observables at the determined ground state

In [20]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -83.165531158643
  - computed part:      -83.165531158643
~ Nuclear repulsion energy (Hartree): 8.149505156227
> Total ground state energy (Hartree): -75.016026002416
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 10.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [18.87913954  0.0  1.9758966]
 
  0: 
  * Electronic dipole moment (a.u.): [19.214152463247  0.000005900872  1.543208866529]
    - computed part:      [19.214152463247  0.000005900872  1.543208866529]
  > Dipole moment (a.u.): [-0.335012923247  -0.000005900872  0.432687733471]  Total: 0.547222380274
                 (debye): [-0.851517835017  -0.000014998519  1.099782415772]  Total: 1.390900422611
 


We can compare the VQE results to the NumPy exact solver and see that they match. 

In [21]:
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -83.165673074386
  - computed part:      -83.165673074386
~ Nuclear repulsion energy (Hartree): 8.149505156227
> Total ground state energy (Hartree): -75.016167918159
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 10.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [18.87913954  0.0  1.9758966]
 
  0: 
  * Electronic dipole moment (a.u.): [19.213986939316  0.0  1.543435185902]
    - computed part:      [19.213986939316  0.0  1.543435185902]
  > Dipole moment (a.u.): [-0.334847399316  0.0  0.432461414098]  Total: 0.54694209521
                 (debye): [-0.851097115189  0.0  1.099207169361]  Total: 1.390188009104
 
